In [1]:
import polars as pl
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from xgboost import XGBRegressor, XGBClassifier
import skopt.space as ss

import sys
sys.path.append("/Users/leon/Desktop/kaggle")
from src.space_titanic.transformer_impl import SpaceTitanicTransformer

In [2]:
df = pl.scan_csv("./data/train.csv")
df.collect().head()

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,bool,str,str,f64,bool,f64,f64,f64,f64,f64,str,bool
"""0001_01""","""Europa""",false,"""B/0/P""","""TRAPPIST-1e""",39.0,false,0.0,0.0,0.0,0.0,0.0,"""Maham Ofracculy""",false
"""0002_01""","""Earth""",false,"""F/0/S""","""TRAPPIST-1e""",24.0,false,109.0,9.0,25.0,549.0,44.0,"""Juanna Vines""",true
"""0003_01""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",58.0,true,43.0,3576.0,0.0,6715.0,49.0,"""Altark Susent""",false
"""0003_02""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",33.0,false,0.0,1283.0,371.0,3329.0,193.0,"""Solam Susent""",false
"""0004_01""","""Earth""",false,"""F/1/S""","""TRAPPIST-1e""",16.0,false,303.0,70.0,151.0,565.0,2.0,"""Willy Santantines""",true


In [18]:
from polars import StringCache
cache = StringCache()
with cache:
	tf = SpaceTitanicTransformer()
	data = tf.fit_transform(df)
	X, y = data[:, :-1], data[:, -1]
	X = X.toarray()
	y = y.toarray().ravel()

cols:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported']
cols before:  ['PassengerId', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'Transported']
did not add y col


In [15]:
sss = ms.StratifiedShuffleSplit(n_splits=1, random_state=42)
for train_index, test_index in sss.split(X, y):
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]
	test_model = XGBClassifier(
		**{
			"n_estimators": 1100,
			"max_depth": 8,
			"learning_rate": 0.3,
			"subsample": 0.85,
			"early_stopping_rounds": 50
		}
	).fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])
test_model.score(X_test, y_test)
print(test_model.predict(X_test))

[0]	validation_0-logloss:0.61365	validation_1-logloss:0.60849
[1]	validation_0-logloss:0.57048	validation_1-logloss:0.56389
[2]	validation_0-logloss:0.54515	validation_1-logloss:0.53750
[3]	validation_0-logloss:0.52838	validation_1-logloss:0.52113
[4]	validation_0-logloss:0.51813	validation_1-logloss:0.51199
[5]	validation_0-logloss:0.50981	validation_1-logloss:0.50310
[6]	validation_0-logloss:0.50481	validation_1-logloss:0.49865
[7]	validation_0-logloss:0.50144	validation_1-logloss:0.49556
[8]	validation_0-logloss:0.49874	validation_1-logloss:0.49291
[9]	validation_0-logloss:0.49696	validation_1-logloss:0.49251
[10]	validation_0-logloss:0.49575	validation_1-logloss:0.49239
[11]	validation_0-logloss:0.49453	validation_1-logloss:0.49086
[12]	validation_0-logloss:0.49331	validation_1-logloss:0.49114
[13]	validation_0-logloss:0.49263	validation_1-logloss:0.49259
[14]	validation_0-logloss:0.49163	validation_1-logloss:0.49278
[15]	validation_0-logloss:0.49106	validation_1-logloss:0.49259
[1

In [13]:
params = {
	"n_estimators": [800, 1100],
	"max_depth": [8],
	"learning_rate": [0.3],
	"subsample": [0.85],
	"early_stopping_rounds": [50]
}

sss = ms.StratifiedShuffleSplit(n_splits=1, random_state=42)
for train_index, test_index in sss.split(X, y):
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fit_params = {
		"eval_set": [(X_train, y_train), (X_test, y_test)]
	}

	model = ms.GridSearchCV(
		estimator=XGBClassifier(),
		param_grid=params,
		cv=2,
		n_jobs=-1
	).fit(X_train, y_train, **fit_params)

[0]	validation_0-logloss:0.61493	validation_1-logloss:0.61155
[0]	validation_0-logloss:0.61714	validation_1-logloss:0.61338
[0]	validation_0-logloss:0.61493	validation_1-logloss:0.61155
[0]	validation_0-logloss:0.61714	validation_1-logloss:0.61338
[1]	validation_0-logloss:0.57440	validation_1-logloss:0.56719
[1]	validation_0-logloss:0.57247	validation_1-logloss:0.56623
[1]	validation_0-logloss:0.57247	validation_1-logloss:0.56623
[1]	validation_0-logloss:0.57440	validation_1-logloss:0.56719
[2]	validation_0-logloss:0.54965	validation_1-logloss:0.54086
[2]	validation_0-logloss:0.54745	validation_1-logloss:0.53824
[2]	validation_0-logloss:0.54745	validation_1-logloss:0.53824
[2]	validation_0-logloss:0.54965	validation_1-logloss:0.54086
[3]	validation_0-logloss:0.53439	validation_1-logloss:0.52280
[3]	validation_0-logloss:0.53328	validation_1-logloss:0.52367
[3]	validation_0-logloss:0.53328	validation_1-logloss:0.52367
[3]	validation_0-logloss:0.53439	validation_1-logloss:0.52280
[4]	vali

In [14]:
model = XGBClassifier(
	**{
		"n_estimators": 1100,
		"max_depth": 8,
		"learning_rate": 0.3,
		"subsample": 0.85,
		# "early_stopping_rounds": 50
	}
).fit(X, y)
model.score(X_test, y_test)

0.8310344827586207

In [19]:
import numpy as np
np.set_printoptions(threshold=np.inf)
with cache:
	xt = pl.scan_csv("./data/test.csv")
	test = tf.transform(xt)
	# pl.scan_csv("./data/train.csv").schema
	res = model.predict(test)
	print(res)

cols:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']
cols before:  ['PassengerId', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP']
added y col
triggered in transform
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [21]:
test_df = pl.scan_csv("./data/test.csv")

solutions = {}
with cache:
	for row in test_df.collect().iter_rows(named=True):
		id = row["PassengerId"]
		x = tf.transform(pl.LazyFrame(row))
		solutions[id] = model.predict(x)

	with open(file="./submission.csv", mode="w") as f:
		f.write("PassengerId,Transported\n")
		for k in solutions.keys():
			f.write(f"{k},{int(solutions[k][0])}\n")

cols:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']
cols before:  ['PassengerId', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP']
added y col
triggered in transform
cols:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']
cols before:  ['PassengerId', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP']
added y col
triggered in transform
cols:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']
cols before:  ['PassengerId', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP']
added y col
triggered in transform
cols:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']
cols before:  ['Passenge